In [191]:
# What i will do here are:
# 1. Create database tracks.db (using SQLite Studio), then create 4 tables mentioned in the READ ME.txt
# 2. Take a look the structure of Library.xml file to figure out :
    # - how to extract informations of one track? 
    # - how to extract each information of a individual track?
# 3. Use the code written in 2 to extract data what we need the insert into database.

In [192]:
######################################################################################################
############################################# START ##################################################
######################################################################################################

In [193]:
# 1. Create database tracks.db (using SQLite Studio), then create 4 tables mentioned in the READ ME.txt

In [194]:
############################################ START 1 #################################################

import sqlite3 as sql

tracks_db_file = r"tracks.db"
xml_file = r"Library.xml"

# i created a database and named it "tracks.db" by using SQLite Studio tool.
# so here write sql scripts to create 4 tables: Artist, Album, Genre and Track
# then I will connect to the database and run those scripts
# after that we will have 4 tables ready for inserting data.

sql_create_tables = '''

DROP TABLE IF EXISTS Artist;
DROP TABLE IF EXISTS Album;
DROP TABLE IF EXISTS Genre;
DROP TABLE IF EXISTS Track;

CREATE TABLE Artist (
    id   INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name TEXT UNIQUE
);

CREATE TABLE Album (
    id        INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id INTEGER,
    title     TEXT UNIQUE
);

CREATE TABLE Genre (
    id       INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name     TEXT UNIQUE
);

CREATE TABLE Track (    
    id       INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    title    TEXT UNIQUE,
    album_id INTEGER,
    genre_id INTEGER,
    length   INTEGER,
    rating   INTEGER,
    count    INTEGER
)

'''

In [195]:
# open the created database
connection = sql.connect(tracks_db_file)

# define cursor variable
curr = connection.cursor()

In [196]:
# run scripts to create tables
curr.executescript(sql_create_tables)

In [197]:
############################################ END 1 #################################################

In [198]:
# 2. Take a look the structure of Library.xml file to figure out :
    # 2.1 how to extract informations of one track? 
    # 2.2 how to extract each information of a individual track?

In [199]:
############################################ START 2 #################################################

In [200]:
# open Library.xml to take a look the structure of data inside
# then we could see the structure likes the below

#<dict>
#   ......
#   <dict>
#       .............
#        <dict>
#          <key>Track ID</key><integer>369</integer>
#          <key>Name</key><string>Another One Bites The Dust</string>
#          <key>Artist</key><string>Queen</string>
#          <key>Composer</key><string>John Deacon</string>
#          <key>Album</key><string>Greatest Hits</string>
#          <key>Genre</key><string>Rock</string>
#          <key>Kind</key><string>MPEG audio file</string>
#          <key>Size</key><integer>4344295</integer>
#          <key>Total Time</key><integer>217103</integer>
#          <key>Disc Number</key><integer>1</integer>
#          <key>Disc Count</key><integer>1</integer>
#          <key>Track Number</key><integer>3</integer>
#           ..............................................
#        </dict>
#        <dict>
#           ..........................
#        </dict>
#        <dict>
#           ..........................
#        </dict>
#        <dict>
#           ..........................
#        </dict>
#   </dict>
#</dict>

In [201]:
# each track's informations in the 3rd <dict> tag
# in order to read the xml file, i'm using The ElementTree XML API

import xml.etree.ElementTree as etree

# parse data inside the file into elements of a tree
tree = etree.parse(xml_file)

# from the tree, we want to extract the content in each 3rd <dict> tag where has track's informations
list_tracks = tree.findall('dict/dict/dict')

# so now we have a list which contains all tracks, each track is an Element object
# each that Element also contains several Elements which are track's informations
# try to print out some sample informations of one track
for i in range(10):
    print(list_tracks[0][i].text)

Track ID
369
Name
Another One Bites The Dust
Artist
Queen
Composer
John Deacon
Album
Greatest Hits


In [202]:
# with the first track from the list, we can see that it has:
# Track ID : 369
# Name     : Another One Bites The Dust
# Artist   : Queen
# Composer : John Deacon
# Album    : Greatest Hits
# and some other informations we have not yet printed out.

# we just want to extract those information (369, "Another One Bites The Dust", "Queen", "John Deacon", "Greatest Hits" ...) 
# not labels (Track ID, Name, Artist, Composer, Album ...)
# so i will write a function to return value of a label of a track

def get_track_infor(track, labelName):
    elements = track.getiterator()
    for item in elements:
        if item.text == labelName:
            return next(elements).text


# print out Name and Artist of the first track
print(get_track_infor(list_tracks[0], "Name"))
print(get_track_infor(list_tracks[0], "Artist"))

Another One Bites The Dust
Queen


In [203]:
# so now i have a list of all track and i also have a function to extract all information of a individual track

In [204]:
############################################ END 2 #################################################

In [205]:
# 3. Use the code written in 2 to extract data what we need the insert into database.

In [206]:
############################################ START 3 ###############################################

In [207]:
# since we need to insert artist_id into table Album, album_id and genre_id into table Track
# but we don't have those infor until we add new records into table Artist, Album and Genre.
# so after adding new records, we need to select new id for the next INSERT sql scripts

In [208]:
# add a new artist
def insert_new_artist(artist_name):
    if artist is not None:
        curr.execute("Insert or Ignore into Artist (name) values (?)", (artist_name, ))
        connection.commit()

In [209]:
# get the artist_id
def get_artist_id(artist_name):
    if artist is not None:
        curr.execute("select id from Artist where name = ? ", (artist_name, ))
        return curr.fetchone()[0]

In [210]:
# add a new album
def insert_new_album(album_name, artist_id):
    if artist_id is not None and album_name is not None:
        curr.execute("Insert or Ignore into Album (artist_id, title) values (?, ?)", (artist_id, album_name))
        connection.commit()

In [211]:
# get the album_id
def get_album_id(album_name):
    if album_name is not None:
        curr.execute("select id from Album where title = ? ", (album_name, ))
        return curr.fetchone()[0]

In [212]:
# add a new genre
def insert_new_genre(genre_name):
    if genre_name is not None:
        curr.execute("Insert or Ignore into Genre (name) values (?)", (genre_name, ))
        connection.commit()

In [213]:
# get the genre_id
def get_genre_id(genre_name):
    if genre_name is not None:
        curr.execute("select id from Genre where name = ? ", (genre_name, ))
        return curr.fetchone()[0]

In [214]:
# loop though each track, i will follow the below sequence to adding records:
# Step 1: add new artist
# Step 2: get artist_id generated by the Step 1
# Step 3: add new album (use the artist_id from the Step 2)
# Step 4: get album_id generated by the Step 3
# Step 5: add new genre
# step 6: get genre_id generated by the Step 5
# step 7: add new track (use album_id from Step 4, genre_id from Step 6)


for one_track in list_tracks:
    
    # extract all informations we need from a individual track
    title = get_track_infor(one_track, "Name")
    album = get_track_infor(one_track, "Album")
    artist = get_track_infor(one_track, "Artist")
    genre = get_track_infor(one_track, "Genre")
    length = get_track_infor(one_track, "Total Time")
    rating = get_track_infor(one_track, "Rating")
    count = get_track_infor(one_track, "Play Count")
    
    # Step 1
    insert_new_artist(artist)
    
    # Step 2
    artist_id = get_artist_id(artist)
    
    # Step 3
    insert_new_album(album, artist_id)
    
    # Step 4
    album_id = get_album_id(album)
    
    # Step 5
    insert_new_genre(genre)
    
    # Step 6
    genre_id = get_genre_id(genre)
    
    # Step 7
    if artist_id is not None and album_id is not None and genre_id is not None:
        curr.execute("Insert or Ignore into Track (title, album_id, genre_id, length, rating, count) values (?, ?, ?, ?, ?, ?)", 
                     (title, album_id, genre_id, length, rating, count))
        connection.commit()    

In [215]:
############################################ END 3 ###############################################